In [ ]:
#problem31: Transitions and Transversions
#transition: A <-> G / C <-> T
#transversion: A <-> C, A<->T, G<->C, G<->T
#transition transversion 변이 처리


data = {}
with open("rosalind_tran.txt") as handle:
    for line in handle:
        line = line.strip()
        if line.startswith(">"):
            id = line[1:]
            data[id] = ""
        else:
            data[id] += line

s1, s2 = data.values()

def trans(s1, s2):
    transisions = 0
    transversions = 0 
    transition_pairs = {('A', 'G'), ('G', 'A'), ('C', 'T'), ('T', 'C') }
    i = 0
    
    while i < len(s1):
        if s1[i] != s2[i]:
            if (s1[i], s2[i]) in transition_pairs:
                transisions += 1
            else:
                transversions += 1
        i += 1
    result = transisions/transversions
    return result

print(f"{trans(s1, s2):.11f}")



2.15000000000


In [ ]:
#problem32: Completing a Tree
# n개의 노드가 있으면 (n-1)개의 간선만 있어야 tree
#사이클이 없어야 한다
# 모든 노드는 연결되어 있어야 한다
#이미 연결된 건 다시 연결하면 안 된다
#머리아파서 못하겠다 너무 어렵다.. 질문이 잘 이해가 안간다..


In [ ]:
#problem33: Catalan Numbers and RNA Secondary Structures
